# 🧠 AI Health & Fitness Plan Generator (Using LangChain)

## 📌 Problem Statement

The goal of this mini project is to build an **AI-powered Health and Fitness Plan Generator** using **LangChain** and a **Large Language Model (LLM)**.

The system takes basic user health information such as age, weight, height, activity level, diet preference, and fitness goal, and generates a **personalized health plan**.

Using a **multi-step LangChain workflow**, the application should:
- Generate a **meal plan** with daily calorie intake and macro breakdown (protein, carbohydrates, fats)
- Generate a **workout plan** including warm-up, main workout, and cool-down
- Combine both outputs into a **weekly fitness strategy** with a schedule and motivational tips

The final output should be displayed clearly inside the Google Colab notebook and exported as a **structured JSON file** (and optional Markdown file) for reuse and evaluation.

This project demonstrates the use of **LangChain orchestration**, **structured output parsing with Pydantic**, and **end-to-end AI application development** in Python.


In [ ]:
!pip install langchain
!pip install langchain-google-genai
!pip install google-generativeai
!pip install tavily-python

In [30]:
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser,PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from tavily import TavilyClient
from pydantic import BaseModel, Field
from typing import Dict, List
from langchain_core.runnables import RunnableParallel
import json

In [13]:
GEMINI_API_KEY=userdata.get("GEMINI_API_KEY")
TAVILY_API_KEY=userdata.get("tavily")


llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=GEMINI_API_KEY)

In [9]:
class MealPlan(BaseModel):
    daily_calories: int = Field(description="Total calories per day")
    protein_g: int
    carbs_g: int
    fats_g: int
    breakfast: str
    lunch: str
    snacks: str
    dinner: str

class WorkoutPlan(BaseModel):
    frequency_per_week: int = Field(description="Number of workout days per week")
    warm_up: str
    main_workout: str
    cool_down: str

class WeeklyStrategy(BaseModel):
    weekly_schedule: Dict[str, str]
    motivation_tips: List[str]


meal_parser = PydanticOutputParser(pydantic_object=MealPlan)
workout_parser = PydanticOutputParser(pydantic_object=WorkoutPlan)
weekly_parser = PydanticOutputParser(pydantic_object=WeeklyStrategy)


In [10]:
meal_prompt = PromptTemplate(
    template="""
You are a certified nutritionist.

User Details:
Age: {age}
Weight (kg): {weight_kg}
Height (cm): {height_cm}
Activity Level: {activity_level}
Diet Preference: {diet_preference}
Fitness Goal: {fitness_goal}

Create a personalized meal plan.

Return output strictly in JSON.
{format_instructions}
""",
    input_variables=[
        "age", "weight_kg", "height_cm",
        "activity_level", "diet_preference", "fitness_goal"
    ],
    partial_variables={
        "format_instructions": meal_parser.get_format_instructions()
    }
)

workout_prompt = PromptTemplate(
    template="""
You are a professional fitness trainer.

User Details:
Age: {age}
Weight (kg): {weight_kg}
Height (cm): {height_cm}
Activity Level: {activity_level}
Fitness Goal: {fitness_goal}

Create a safe and effective workout plan.

Return output strictly in JSON.
{format_instructions}
""",
    input_variables=[
        "age", "weight_kg", "height_cm",
        "activity_level", "fitness_goal"
    ],
    partial_variables={
        "format_instructions": workout_parser.get_format_instructions()
    }
)

weekly_prompt = PromptTemplate(
    template="""
You are a health coach.

Meal Plan:
{meal_plan}

Workout Plan:
{workout_plan}

Combine both into a weekly strategy.
Include day-wise guidance and motivation tips.

Return output strictly in JSON.
{format_instructions}
""",
    input_variables=["meal_plan", "workout_plan"],
    partial_variables={
        "format_instructions": weekly_parser.get_format_instructions()
    }
)



In [19]:
meal_chain = meal_prompt | llm | meal_parser
workout_chain = workout_prompt | llm | workout_parser

parallel_chain = RunnableParallel(
    {
        "meal_plan": meal_chain,
        "workout_plan": workout_chain
    }
)


weekly_chain = weekly_prompt | llm | weekly_parser


In [32]:
user_input = {
    "age": 28,
    "weight_kg": 72,
    "height_cm": 175,
    "activity_level": "moderate",
    "diet_preference": "vegetarian",
    "fitness_goal": "fat loss"
}

parallel_output = parallel_chain.invoke(user_input)

final_output = weekly_chain.invoke({
    "meal_plan": parallel_output["meal_plan"].model_dump(),
    "workout_plan": parallel_output["workout_plan"].model_dump()
})




In [41]:
final_plan = {
    "meal_plan":parallel_output["meal_plan"].model_dump(),
    "workout_plan":parallel_output["workout_plan"].model_dump(),
    "weekly_strategy": final_output.model_dump()
}

with open("final_plan.json", "w") as f:
    json.dump(final_plan, f, indent=4)

print(json.dumps(final_plan, indent=2))

{
  "meal_plan": {
    "daily_calories": 2150,
    "protein_g": 149,
    "carbs_g": 279,
    "fats_g": 52,
    "breakfast": "High-Protein Greek Yogurt Bowl: 1 cup (200g) plain Greek yogurt, 1/2 cup mixed berries, 1 tbsp pumpkin seeds, 1/2 cup (cooked) rolled oats.",
    "lunch": "Hearty Lentil & Chickpea Salad with Brown Rice: 1 cup (cooked) lentils, 1/2 cup (cooked) chickpeas, large portion of mixed greens and vegetables (cucumber, tomato, bell peppers), dressed with 1 tbsp olive oil and lemon juice. Served with 1 cup (cooked) brown rice.",
    "snacks": "Mid-morning: 1 large apple. Mid-afternoon: Protein shake (1.5 scoops plant-based protein powder with water or unsweetened almond milk) and 1/4 cup almonds.",
    "dinner": "Tofu & Vegetable Stir-fry with Brown Rice: 200g firm tofu (cubed and grilled/pan-fried), large portion of mixed stir-fried vegetables (e.g., broccoli, bell peppers, carrots, mushrooms, snow peas), seasoned with soy sauce/tamari and 1 tsp sesame oil. Served with 1.